In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt


In [ ]:

#import surface file, such as lh.pial or lh.white
#this is an example I can use

vertices, faces = nib.freesurfer.read_geometry("bert/surf/lh.pial")
print(vertices.shape)

annot = nib.freesurfer.io.read_annot("bert/label/lh.aparc.annot",orig_ids = False)
#axis 0 contains the labels no.
#axis 1 contains the coordinates
#axis 2 contains the label name (str)

label= annot[0]
ctab= annot[1]
name = (annot[2])
for i in range(len(name)):
    name[i]= name[i].decode(encoding="utf-8", errors="strict")

print(label, name)

In [ ]:

#just making sure the file is not corrupted 
print(len(label),len(vertices), len(ctab))
print(np.any(vertices))
if np.any(vertices)== True:
    print("File fine")
else:
    print("no vertices found ")


In [ ]:
#enter ROI label number, refer to the DK_atlas_ref text 
# note that the corpus callosum (4), contains no data
ROI_label= 34
ROI_name= name[ROI_label]

ROI= np.where(label==ROI_label)[0]
print (ROI, ROI_name)

#extract the vertices of the ROI
ROI_vertices=vertices[ROI]


In [ ]:
# counting regular voxels that contain data, to ensure it contains a volume
# this gives us the same number of voxels as size-1 boxes
# a for loop is slightly inneficient, but gets the job done for smaller ROI

counter= 0

for i in ROI_vertices:
    counter += 1

print(ROI_vertices.shape)
print (counter)


In [ ]:
x_min = int(np.min(ROI_vertices[:,0]))
x_max = int(np.max(ROI_vertices[:,0])) +1
y_min = int(np.min(ROI_vertices[:,1]))
y_max = int(np.max(ROI_vertices[:,1])) +1
z_min = int(np.min(ROI_vertices[:,2])) 
z_max = int(np.max(ROI_vertices[:,2])) +1


def box_counting(box_size):
    x_pos= np.arange(x_min, x_max, box_size)
    y_pos = np.arange(y_min, y_max, box_size)
    z_pos= np.arange(z_min, z_max, box_size)
    counter=0
    for x in x_pos:
        for y in y_pos:
            for z in z_pos:
                cube = ((ROI_vertices[:,0]>= x) & (ROI_vertices[:,0]< x+box_size) & (ROI_vertices[:,1]>= y) & (ROI_vertices[:,1]< y+box_size) & (ROI_vertices[:,2]>= z) & (ROI_vertices [:,2]< z+box_size))
                if np.any(cube):
                    counter+=1


    return counter



In [ ]:
#calculates from box sizes of factors of 240
x= np.zeros(18)
y= np.zeros(18)
idx=0
for i in range(2,240):
    if 240 % i == 0:
        number_of_boxes = box_counting(i)
        box_size = i
        # x is the x axis of our graph, representing box size
        x[idx] = int(box_size)
        y[idx]= int(number_of_boxes)
        idx += 1
x = x.astype(int)
y = y.astype(int)   
y=y[0:idx]
x=x[0:idx]


In [ ]:
#calculates from box sizes of factors of 256
y = np.zeros(40)
x=np.zeros(40)
idx = 0
#for i in range(padded_shape):
for i in range(1,11):
        box_size=int(2**i)
        num_box= box_counting(box_size)
        y[idx]= num_box
        x[idx]= box_size
        idx += 1
x = x.astype(int)
y = y.astype(int)   
y=y[0:idx]
x=x[0:idx]

#ultimately, I used factors of 240

In [ ]:
print(x, y)


In [ ]:
log_x= np.log(x)
log_y=np.log(y)
print(log_x)
print(log_y)

In [ ]:



plt.plot(x,y,color='orange')
plt.title('rel. volume against magnification')
plt.xlabel("cube size(voxels)")
plt.ylabel('number of cubes')
plt.show()


In [ ]:
#finding the "ideal" fractal dimension range, systematically
# selects the widest range with a minimum of 4 box sizes for which the r2 > 0.99

widest_range = 0 
best_fd = 0
best_r2= 0
stdev_fd= 0
fd_array = np.array([0])
for j in range(0,idx+1):
    for k in range(1,idx+1):
        if k>j:
            rf_log_x = log_x[j:k]
            rf_log_y = log_y[j:k]
            m, c = np.polyfit(rf_log_x, rf_log_y, 1)
            fd = -m
            correlation_coefficient = np.corrcoef(rf_log_x, rf_log_y)[0,1]
            r2 = correlation_coefficient**2
            if  0.995 < r2 < 1:
                if  k-j >= widest_range:
                    widest_range = k - j
                    min_j = j
                    max_k = k-1
                    best_fd = fd
                    best_r2 = r2    
                    fd_array = np.append(fd_array, fd)
    stdev= np.std(fd_array)

In [ ]:
print("the best estimate fractal dimension is: ", best_fd)
print("with R-squared value: ", best_r2, "and standard dev:", stdev)
print( "ideal range is box sizes", x[min_j], "to", x[max_k])
if widest_range<4:
    print('warning: range too small, proceed with caution')
#keep in mind this does not consider multi- or bi-fractals (yet), so the FD with the highest r2 will be chosen

In [ ]:
#use this to explore how the log-log graph changes at different ranges, with a line of best fit!
rf_log_x = log_x[0:8]
rf_log_y = log_y[0:8]
plt.plot(rf_log_x,rf_log_y,color='blue',marker='o')
plt.title('logS against logN')
plt.xlabel("log(S)")
plt.ylabel('log(N)')
m, c = np.polyfit(rf_log_x, rf_log_y, 1)
plt.plot(rf_log_x, m*rf_log_x + c, color='red')  # Plot
plt.show()

In [ ]:
print("Fractal Dimension is: ", -m)
correlation_coefficient = np.corrcoef(rf_log_x, rf_log_y)[0, 1]
r2 = correlation_coefficient**2
print("R-squared value is: ", r2)